In [12]:
:dep darn
:dep ndarray
:dep ndarray-csv
:dep polars = { version = "x", features = ["csv","describe","fmt","lazy","ndarray"]}
:dep plotters = {version = "0.3.5", features = ["evcxr", "line_series"]}
    
extern crate csv;

use std::io::prelude::*;
use std::fs::*;
use std::path::Path
use ndarray::prelude::*;
use ndarray_csv::Array2Reader;

use polars::prelude::*;
use std::fs::File;

extern crate plotters;
use plotters::prelude::*;
use plotters::series::*;

In [13]:
//Load CSV datafram
let mut database_rd = csv::Reader::from_path(Path::new("./db/credit_data.csv"))?; 
//Convert csv to multidimension array
let data: Array2<String> = database_rd.deserialize_array2_dynamic()?;
//show frame on html table format
darn::show_frame(&data,None);



Error: mismatched types

In [15]:
//load from polars
let data_frame_polars = CsvReader::from_path("./db/credit_data.csv").unwrap()
    .has_header(true)
    .finish().unwrap();

//print dataframe
println!("{:?}",data_frame_polars);
//header data
println!("{}",data_frame_polars.head(Some(2)));
//tail data
println!("{}",data_frame_polars.tail(Some(5)));
//filter data
let filter = data_frame_polars.clone().lazy().filter(col("income").gt_eq(1.377630)).collect();
println!("{:?}",filter);





shape: (2_000, 5)
┌──────────┬──────────────┬───────────┬─────────────┬─────────┐
│ clientid ┆ income       ┆ age       ┆ loan        ┆ default │
│ ---      ┆ ---          ┆ ---       ┆ ---         ┆ ---     │
│ i64      ┆ f64          ┆ f64       ┆ f64         ┆ i64     │
╞══════════╪══════════════╪═══════════╪═════════════╪═════════╡
│ 1        ┆ 66155.925095 ┆ 59.017015 ┆ 8106.532131 ┆ 0       │
│ 2        ┆ 34415.153966 ┆ 48.117153 ┆ 6564.745018 ┆ 0       │
│ 3        ┆ 57317.170063 ┆ 63.108049 ┆ 8020.953296 ┆ 0       │
│ 4        ┆ 42709.534201 ┆ 45.751972 ┆ 6103.64226  ┆ 0       │
│ 5        ┆ 66952.688845 ┆ 18.584336 ┆ 8770.099235 ┆ 1       │
│ …        ┆ …            ┆ …         ┆ …           ┆ …       │
│ 1996     ┆ 59221.044874 ┆ 48.518179 ┆ 1926.729397 ┆ 0       │
│ 1997     ┆ 69516.127573 ┆ 23.162104 ┆ 3503.176156 ┆ 0       │
│ 1998     ┆ 44311.449262 ┆ 28.017167 ┆ 5522.786693 ┆ 1       │
│ 1999     ┆ 43756.056605 ┆ 63.971796 ┆ 1622.722598 ┆ 0       │
│ 2000     ┆ 69436.579

In [161]:
//unique data
let df= data_frame_polars.clone().unique(Some(&["default".to_string()]),UniqueKeepStrategy::First, None);
println!("{:?}",df);

let df= data_frame_polars.clone().group_by(["default"])?.select(["default"]).count();
println!("{:?}",df);

let ndarray = df.unwrap().to_ndarray::<Float64Type>(IndexOrder::Fortran).unwrap();
// let tuples: Vec<(f64, f64)> = ndarray
//     .rows()
//     .into_iter()
//     .map(|row| (row[0], row[1]))
//     .collect();


evcxr_figure((640, 480), |root| {
    
 let mut chart = ChartBuilder::on(&root)
        .x_label_area_size(35)
        .y_label_area_size(40)
        .margin(5)
        .caption("Count Test", ("sans-serif", 50.0))
        .build_cartesian_2d(0u32..2u32, 0f64..2000f64)?;  
    
 chart.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .y_labels(5)
        .x_label_formatter(&|x| format!("{}", *x))
        .y_label_formatter(&|y| format!("{}", *y))
        .draw()?;


 chart.draw_series(
        Histogram::vertical(&chart)
            .style(RED.mix(0.5).filled())
             .margin(4)
            .data(ndarray
                .rows()
                .into_iter()
                .map(|row|((row[0]) as u32, row[1]))))?;

    
    Ok(())
}).style("width:100%")

Ok(shape: (2, 5)
┌──────────┬──────────────┬───────────┬─────────────┬─────────┐
│ clientid ┆ income       ┆ age       ┆ loan        ┆ default │
│ ---      ┆ ---          ┆ ---       ┆ ---         ┆ ---     │
│ i64      ┆ f64          ┆ f64       ┆ f64         ┆ i64     │
╞══════════╪══════════════╪═══════════╪═════════════╪═════════╡
│ 1        ┆ 66155.925095 ┆ 59.017015 ┆ 8106.532131 ┆ 0       │
│ 5        ┆ 66952.688845 ┆ 18.584336 ┆ 8770.099235 ┆ 1       │
└──────────┴──────────────┴───────────┴─────────────┴─────────┘)
Ok(shape: (2, 2)
┌─────────┬───────────────┐
│ default ┆ default_count │
│ ---     ┆ ---           │
│ i64     ┆ u32           │
╞═════════╪═══════════════╡
│ 0       ┆ 1717          │
│ 1       ┆ 283           │
└─────────┴───────────────┘)


Count Test
 
 
 
0
 
 
 
500
 
 
 
1000
 
 
 
1500
 
 
 
2000
 
 
 
 
0
 
 
 
1
 
 
 
2

In [169]:

let data_frame_polars = CsvReader::from_path("./db/credit_data.csv").unwrap()
    .has_header(true)
    .finish().unwrap();


let df= data_frame_polars.clone().lazy().select([col("age").cast(DataType::Int32).alias("cast_age")]).with_columns([col("cast_age").fill_null(lit(0))]).collect();
println!("{:?}",df);
let group= df?.group_by(["cast_age"])?.select(["cast_age"]).count()?;
println!("{:?}",group);

let min= group.clone().column("cast_age")?.min::<i32>().unwrap().unwrap();
let max= group.clone().column("cast_age")?.max::<i32>().unwrap().unwrap();

let minY= group.clone().column("cast_age_count")?.min::<i32>().unwrap().unwrap();
let maxY= group.clone().column("cast_age_count")?.max::<i32>().unwrap().unwrap();

let ndarray = group.clone().to_ndarray::<Int32Type>(IndexOrder::Fortran).unwrap();

evcxr_figure((640, 480), |root| {
    
 let mut chart = ChartBuilder::on(&root)
        .x_label_area_size(35)
        .y_label_area_size(40)
        .margin(5)
        .caption("Count age", ("sans-serif", 10.0))
        .build_cartesian_2d(min..max, minY..maxY)?;  
    
 chart.configure_mesh()
        .x_label_formatter(&|x| format!("{}", *x))
        .y_label_formatter(&|y| format!("{}", *y))
        .draw()?;


 chart.draw_series(
        Histogram::vertical(&chart)
            .style(RED.mix(0.5).filled())
             .margin(0)
            .data(ndarray
                .rows()
                .into_iter()
                .map(|row|(row[0], row[1]  ))))?;

    
    Ok(())
}).style("width:100%")

Ok(shape: (2_000, 1)
┌──────────┐
│ cast_age │
│ ---      │
│ i32      │
╞══════════╡
│ 59       │
│ 48       │
│ 63       │
│ 45       │
│ 18       │
│ …        │
│ 48       │
│ 23       │
│ 28       │
│ 63       │
│ 56       │
└──────────┘)
shape: (50, 2)
┌──────────┬────────────────┐
│ cast_age ┆ cast_age_count │
│ ---      ┆ ---            │
│ i32      ┆ u32            │
╞══════════╪════════════════╡
│ 27       ┆ 54             │
│ 21       ┆ 44             │
│ 24       ┆ 40             │
│ 0        ┆ 3              │
│ 30       ┆ 33             │
│ …        ┆ …              │
│ 59       ┆ 38             │
│ 62       ┆ 45             │
│ 38       ┆ 34             │
│ 47       ┆ 53             │
│ -36      ┆ 1              │
└──────────┴────────────────┘


Count age
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
10
 
 
 
20
 
 
 
30
 
 
 
40
 
 
 
50
 
 
 
60
 
 
 
 
-40
 
 
 
-20
 
 
 
0
 
 
 
20
 
 
 
40
 
 
 
60

In [213]:

let data_frame_polars = CsvReader::from_path("./db/credit_data.csv").unwrap()
    .has_header(true)
    .finish().unwrap();


let df= data_frame_polars.clone().lazy()
    .select([col("income").cast(DataType::Int32).alias("cast_income") / lit(1000)])
    .with_columns([col("cast_income").fill_null(lit(0))]).collect();

println!("{:?}",df);
let group= df?.group_by(["cast_income"])?.select(["cast_income"]).count()?;
println!("{:?}",group);

let min= group.clone().column("cast_income")?.min::<i32>().unwrap().unwrap();
let max= group.clone().column("cast_income")?.max::<i32>().unwrap().unwrap();

let minY= group.clone().column("cast_income_count")?.min::<i32>().unwrap().unwrap();
let maxY= group.clone().column("cast_income_count")?.max::<i32>().unwrap().unwrap();

let ndarray = group.clone().to_ndarray::<Int32Type>(IndexOrder::Fortran).unwrap();

evcxr_figure((640, 480), |root| {
    
 let mut chart = ChartBuilder::on(&root)
        .x_label_area_size(35)
        .y_label_area_size(40)
        .margin(5)
        .caption("Count icome", ("sans-serif", 10.0))
        .build_cartesian_2d(min..max, minY..maxY)?;  
    
 chart.configure_mesh()
        .x_label_formatter(&|x| format!("{}", *x * 1000))
        .y_label_formatter(&|y| format!("{}", *y))
        .draw()?;


 chart.draw_series(
        Histogram::vertical(&chart)
            .style(RED.mix(0.5).filled())
             .margin(0)
            .data(ndarray
                .rows()
                .into_iter()
                .map(|row|(row[0], row[1]  ))))?;

    
    Ok(())
}).style("width:100%")

Ok(shape: (2_000, 1)
┌─────────────┐
│ cast_income │
│ ---         │
│ i32         │
╞═════════════╡
│ 66          │
│ 34          │
│ 57          │
│ 42          │
│ 66          │
│ …           │
│ 59          │
│ 69          │
│ 44          │
│ 43          │
│ 69          │
└─────────────┘)
shape: (50, 2)
┌─────────────┬───────────────────┐
│ cast_income ┆ cast_income_count │
│ ---         ┆ ---               │
│ i32         ┆ u32               │
╞═════════════╪═══════════════════╡
│ 30          ┆ 44                │
│ 27          ┆ 41                │
│ 24          ┆ 39                │
│ 21          ┆ 34                │
│ 63          ┆ 34                │
│ …           ┆ …                 │
│ 38          ┆ 33                │
│ 62          ┆ 49                │
│ 65          ┆ 38                │
│ 53          ┆ 40                │
│ 68          ┆ 39                │
└─────────────┴───────────────────┘


Count icome
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
30
 
 
 
35
 
 
 
40
 
 
 
45
 
 
 
50
 
 
 
55
 
 
 
 
20000
 
 
 
25000
 
 
 
30000
 
 
 
35000
 
 
 
40000
 
 
 
45000
 
 
 
50000
 
 
 
55000
 
 
 
60000
 
 
 
65000